In [19]:
from lxml import etree
import json
import time
import os
import cv2
import math

import pandas as pd
import numpy as np

In [20]:
from shapely.geometry import MultiPoint, Polygon, LineString

In [21]:
json_path = 'D:\\datasets\\nut\\trainval.json'
image_path = 'D:\\datasets\\nut\\images\\'
target_path = 'D:\\datasets\\nut\\odtk\\'

In [22]:
with open(json_path,'r') as f:
    json_dict = json.load(f)

In [24]:
#calc_bearing是一个用arctan求θ的简单函数。你必须把函数封装起来以确保w和h是正的，并且theta在-pi/2到pi/2或者-pi到pi的范围内。
def calc_bearing(pointA, pointB):
    delta_x = pointB[0]-pointA[0]
    delta_y = pointB[1]-pointA[1]
    bearing = math.atan2(delta_y,delta_x)
    return bearing

def _corners2rotatedbbox(corners):
    centre = np.mean(np.array(corners), 0)
    theta = calc_bearing(corners[0], corners[1])
    rotation = np.array([[np.cos(theta), -np.sin(theta)],
                        [np.sin(theta), np.cos(theta)]])
    out_points = np.matmul(corners - centre, rotation) + centre
    x, y = list(out_points[0,:])
    w, h = list(out_points[2, :] - out_points[0, :])
    return [x, y, w, h, theta]

anno_list = json_dict['annotations']

for i in anno_list:
    seg = i['segmentation']
    cnt = np.reshape(np.float32(seg),(-1,2))
    mp = MultiPoint(cnt)
    rect = mp.minimum_rotated_rectangle
    x_list = rect.boundary.xy[0][:4]
    y_list = rect.boundary.xy[1][:4]
    corners = np.vstack((x_list,y_list)).T
    rbox = _corners2rotatedbbox(corners)
    i['bbox'] = list(rbox)
    i['segmentation'] = list(corners)

In [25]:
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(NpEncoder, self).default(obj)

In [26]:
with open('D:\\datasets\\nut\\rbox_trainval.json','w') as f:
    json.dump(json_dict,f, cls=NpEncoder)